# AI Project - Time Table using Genetic Algorithms

Group Members:

Zaynab Batool Reza 

M. Adil Fayyaz 

**ASSUMPTIONS**

1. The exams can only be scheduled in either the morning session (Starting at 9 am) or in the evening session (Starting at 2 pm)
2. Each exam lasts for 3 hours
3. There will be more than one exam schedules for courses with more than 28 students 
4. Half of the teachers will invigilate the 9 am session and the other half will invigilate the 2 pm session
5. Consecutive exam means if one exam is at 9 am its consecutive exam is at 2 pm


Import libraries

In [ ]:
import numpy as np
import pandas as pd
import csv
import random
import copy

## **Importing Larger DataSet (790 rows)**

In [ ]:
# Import the teachers.csv file
from google.colab import files
uploaded = files.upload()
f = open('teachers.csv')
teachersDF = pd.read_csv(f,header= None)
teachersDF.columns = ['TeacherName']
teachersDF


Saving teachers.csv to teachers.csv


,TeacherName
0,Ayesha Bano
1,Aqeel Shahzad
2,Farah Naz
3,Hamda Khan
4,Usman Rashid
...,...
58,Kashif Munir
59,Hammad Majeed
60,Noreen Jamil
61,Hassan Mustafa


In [ ]:
# Import the courses.csv
uploaded = files.upload()
f = open('courses.csv')
coursesDF=pd.read_csv(f,header = None)
coursesDF.columns = ['CourseCode','CourseName']
# Fetch the unique rows only
duplicateRowsDF = coursesDF[coursesDF.duplicated(['CourseCode'])]
coursesDF.drop_duplicates(subset=['CourseCode'], keep='first',inplace=True)
coursesDF.reset_index(drop=True, inplace=True)
coursesDF

Saving courses.csv to courses.csv


,CourseCode,CourseName
0,CS217,Object Oriented Programming
1,EE227,Digital Logic Design
2,CS211,Discrete Structures
3,SE110,Intro to Software Engineering
4,CS118,Programming Fundamentals
5,CS219,Database Systems
6,CS220,Operating Systems
7,CS302,Design & Analysis of Algorithms
8,CY2012,Digital Forensics
9,CS307,Computer Networks


In [ ]:
# Import the studentCourse.csv file
uploaded = files.upload()
f = open('studentCourse.csv')
studentCourse=pd.read_csv(f)
studentCourse.columns = ['Index', 'StudentName', 'CourseCode']
# Fetch the unique rows only
duplicateRowsDF = studentCourse[studentCourse.duplicated(['StudentName', 'CourseCode'])]
studentCourse.drop_duplicates(subset=['StudentName', 'CourseCode'], keep='first',inplace=True)
studentCourse.reset_index(drop=True, inplace=True)
studentCourse


Saving studentCourse.csv to studentCourse.csv


,Index,StudentName,CourseCode
0,NaN,Mohammed Azam,CS217
1,NaN,Mohamed F Bakaram,CS217
2,NaN,Manal Aldaihani,CS217
3,NaN,Ibrar Manawer,CS217
4,NaN,Adam Flowers,CS217
...,...,...,...
785,NaN,Naveed Hussain,MT205
786,NaN,Sarah L Clark,MT205
787,NaN,Maria Lytras,MT205
788,NaN,Bibi A Khan,MT205


In [ ]:
# Import the studentNames.csv file
uploaded = files.upload()
f = open('studentNames.csv')
studentNames=pd.read_csv(f, header= None)
studentNames.columns = ['StudentName']
studentNames


Saving studentNames.csv to studentNames.csv


,StudentName
0,Mohammed Azam
1,Mohamed F Bakaram
2,Manal Aldaihani
3,Ibrar Manawer
4,Adam Flowers
...,...
195,Naveed Hussain
196,Sarah L Clark
197,Maria Lytras
198,Bibi A Khan


Function to convert day number to **day**

In [ ]:
def getDay(num):
  if num%5==0:
    return "Friday"
  elif num==1 or num==6:
    return "Monday"
  elif num==2 or num==7:
    return "Tuesday"
  elif num==3 or num==8:
    return "wednesday"
  elif num==4 or num==9:
    return "Thursday"
  else:
    return num

## **Classes Declarations**

### Time Class

Attributes:
1. Day - The day number of the exam
2. Starting Hour - Exam at 9 am or 2 pm
3. Week - The week number of the exam

In [ ]:
class Time:
  def __init__(self, day, starting_hour):
    self.day = day
    self.starting_hour = starting_hour
    self.week=(day//5)+1
  def __repr__(self):
    return ('(Week {2}, Day {0},Time {1})'.format(getDay(self.day), self.starting_hour, self.week))

### Exam Class

Attributes:
1. Invigilator - The name of teacher assigned to an exam session
2. Course Code - The course code of the course
3. Time - The time object
4. Room - The classroom number
5. MultipleRooms - Number of additional rooms needed for a course

In [ ]:
class Exam:
  def __init__(self, invigilator, courseCode, time, room, multipleRooms):
    self.invigilator = invigilator
    self.courseCode = courseCode
    self.time = time
    self.room = room
    self.multipleRooms = multipleRooms

  def __repr__(self):
    return ('(Invigilator: {0},Course Code: {1},Time: {2},Room: {3}, Multiple Rooms: {4})'.format(self.invigilator, self.courseCode, self.time, self.room, self.multipleRooms))

  

### Class Individual

Attributes:
1. Chromosome - the binary encoded string representing a schedule
2. Fitness - a float value that represents the fitness of the schedule
3. h1 - h6 : the hard constraints values
4. s1 - s4 : the soft constraints values

In [ ]:
class Individual:
  def __init__(self, chromosome, fitness, h1=0, h2=0, h3=0, h4=0, h5=0, h6=0, s1=0, s2=0, s3=0, s4=0):
    self.chromosome=chromosome
    self.fitness=fitness
    self.h1 = h1
    self.h2 = h2
    self.h3 = h3
    self.h4 = h4
    self.h5 = h5
    self.h6 = h6
    self.s1 = s1
    self.s2 = s2
    self.s3 = s3
    self.s4 = s4

  def __repr__(self):
    return ('(Chromosome: {0}, Fitness: {1})'.format(self.chromosome, self.fitness))
  

## **Binary Encoding of Entire DataSet**

Create classrooms and assigning binary values to them

In [ ]:
#create dataframe for classrooms
classeslist=[]
binaryvalues=[]
start=101
for i in range(10):
  classeslist.append("c"+str(start))
  binaryvalues.append('{0:07b}'.format(i)) #get binary strings
  start+=1

classes = pd.DataFrame(list(zip(classeslist, binaryvalues)),
               columns =['Class', 'Binary'])

classes

,Class,Binary
0,c101,0000000
1,c102,0000001
2,c103,0000010
3,c104,0000011
4,c105,0000100
5,c106,0000101
6,c107,0000110
7,c108,0000111
8,c109,0001000
9,c110,0001001


Assigning binary values to students, teachers and courses

In [ ]:
#assign binary values to teachers, courses and students
def getBinaryValues(students, teachers, courses):
  studentbinary=[]
  teacherbinary=[]
  coursebinary=[]

  for i in range(len(students)):
    studentbinary.append('{0:08b}'.format(i)) #get binary strings

  for i in range(len(teachers)):
    teacherbinary.append('{0:07b}'.format(i)) #get binary strings

  for i in range(len(courses)):
    coursebinary.append('{0:07b}'.format(i)) #get binary strings

  #adding these as columns to the dataframes
  students['Binary']=studentbinary
  teachers['Binary']=teacherbinary
  courses['Binary']=coursebinary

getBinaryValues(studentNames, teachersDF, coursesDF)

teachersDF



,TeacherName,Binary
0,Ayesha Bano,0000000
1,Aqeel Shahzad,0000001
2,Farah Naz,0000010
3,Hamda Khan,0000011
4,Usman Rashid,0000100
...,...,...
58,Kashif Munir,0111010
59,Hammad Majeed,0111011
60,Noreen Jamil,0111100
61,Hassan Mustafa,0111101


Split the teachers into two groups

In [ ]:
#function to divide teachers into two halves

def divideTeachers():
  teachersA=pd.DataFrame(columns = ['TeacherName', 'Binary'])
  teachersB=pd.DataFrame(columns = ['TeacherName', 'Binary'])
  division=len(teachersDF)//2
  for i in range(division):
    teachersA=teachersA.append({'TeacherName': teachersDF.at[i,'TeacherName'], 'Binary': teachersDF.at[i,'Binary']}, ignore_index=True)
  for i in range(division,len(teachersDF)):
    teachersB=teachersB.append({'TeacherName': teachersDF.at[i,'TeacherName'], 'Binary': teachersDF.at[i,'Binary']}, ignore_index=True)
  return teachersA, teachersB
  
teachersA, teachersB= divideTeachers()
teachersB

,TeacherName,Binary
0,Mehreen Alam,0011111
1,Mehwish Hassan,0100000
2,Shams Farooq,0100001
3,Ameen Chilwan,0100010
4,Shafaq Riaz,0100011
5,Khadija Farooq,0100100
6,Amna Irum,0100101
7,Irum Inayat,0100110
8,Bilal Khalid,0100111
9,Shehreyar Rashid,0101000


### Representation of Exams and Schedules as Binary Strings

Generate a new gene or random Exam to insert in the initial population

In [ ]:
#generate a string of binary values to represent one exam
#class, day, startinghour, course, teacher

def getRandomExam(classes, courses, teachers):
  

  classroom=random.choice(classes['Binary'].tolist())
  day= '{0:07b}'.format(random.randint(1,10) )
  #print(int(day, 2))
  startinghour= '{0:07b}'.format(random.choice([9,14])) 
  
  course=random.choice(courses['Binary'].tolist())
  if (int(startinghour, 2)==9): #choose teachers from first half   
    teacher=random.choice(teachersA['Binary'].tolist())
  elif (int(startinghour,2)==14): #choose teachers from second half
    teacher=random.choice(teachersB['Binary'].tolist())

  courseCode=coursesDF.at[list(np.where(coursesDF['Binary']==course)[0])[0],'CourseCode']
  studentsCount = 0
  studentsCount = list(np.where(studentCourse['CourseCode'] == courseCode)[0])
  studentsCount2 = len(studentsCount)
  if studentsCount2 > 28:
    # print(studentsCount2)
    multipleExam = '{0:07b}'.format(studentsCount2 // 28)
  else:
    multipleExam = '{0:07b}'.format(0)
  
  exam=classroom+day+startinghour+course+teacher+multipleExam
  return exam

print("7 bits for class, 7 bits for day, 7 bit for starting hour, 7 bits for course, 7 bits for teacher, 7 bits for multiple rooms")
exam=getRandomExam(classes, coursesDF, teachersDF)
print(exam) 


7 bits for class, 7 bits for day, 7 bit for starting hour, 7 bits for course, 7 bits for teacher, 7 bits for multiple rooms
000001000001100001110001011001101010000001


Function used by mutation function, to randomly generate a new exam with the same course and number of rooms

In [ ]:
#given an exam, randomly generate its teacher, classroom, starting hour and day
def convertToChromosome(exam):

  classroom=random.choice(classes['Binary'].tolist())
  day= '{0:07b}'.format(random.randint(1,10) )
  startinghour= '{0:07b}'.format(random.choice([9,14]))
  course=exam[21:28]
  if (int(startinghour, 2)==9): #choose teachers from first half   
    teacher=random.choice(teachersA['Binary'].tolist())
  elif (int(startinghour,2)==14): #choose teachers from second half
    teacher=random.choice(teachersB['Binary'].tolist())
  courseCode=coursesDF.at[list(np.where(coursesDF['Binary']==course)[0])[0],'CourseCode']
  multipleExam = exam[35:42]

  exam=classroom+day+startinghour+course+teacher+multipleExam
  return exam


Convert the binary encoded bits to objects

In [ ]:
#to convert chromosome back to Exam form
def convertToExamObject(exam):

  #0-7 bits for class, 7-14 bits for day, 14-21 bit for starting hour, 21-28 bits for course, 28-35 bits for teacher, 35-42 bits for multiple rooms
  #check if all binary values are valid
  if (len(list(np.where(classes['Binary']==exam[0:7])[0]))<=0 or len(list(np.where(coursesDF['Binary']==exam[21:28])[0]))<=0 or len(list(np.where(teachersDF['Binary']==exam[28:35])[0]))<=0 or int(exam[7:14], 2)>10 or int(exam[7:14], 2)<1):
    return None
  classroom=classes.at[list(np.where(classes['Binary']==exam[0:7])[0])[0],'Class']
  day=int(exam[7:14], 2)
  time=int(exam[14:21], 2)
  course=coursesDF.at[list(np.where(coursesDF['Binary']==exam[21:28])[0])[0],'CourseCode']
  teacher=teachersDF.at[list(np.where(teachersDF['Binary']==exam[28:35])[0])[0],'TeacherName']
  multipleExam = int(exam[35:42],2)

  return Exam(teacher, course, Time(day, time), classroom, multipleExam)







Given a schedule splits it into the individual exams and returns a list of them

In [ ]:
#function to get individual exam strings from schedule
def getExamStrings(schedule):
  exams=[]
  i=0
  while (i<len(schedule)):
    exams.append(schedule[i:i+42]) #here 42 is total number of bits in each exam, change if adding more bits to starting hour
    i=i+42
  return exams

## **Utility Functions needed for GA Algorithm**

Generate the initial population using the size and dataframes

In [ ]:
def generatePopulation(populationSize, courses, classes, teacher):
  population = []
  for x in range(populationSize):
    schedule=[]
    CourseListCount = [0 for _ in range(len(courses))]
    #0-7 bits for class, 7-14 bits for day, 14-21 bit for starting hour, 21-28 bits for course, 28-35 bits for teacher, 35-42 bits for multiple rooms
    for i in range(len(courses)):
      duplicate=True
      while(duplicate==True):
        temp=getRandomExam(classes, courses, teacher)
        if temp[21:28] not in [j[21:28] for j in schedule] : #if this course has not been generated before
          if temp[35:42] == '0000000':
            duplicate=False
            # Complete 
            CourseListCount[i] = 1
          else:
            while CourseListCount[i] < int(temp[35:42],2):
              temp2=getRandomExam(classes, courses, teacher)
              if temp2[21:28] == temp[21:28]:
                schedule.append(temp2)
                CourseListCount[i] += 1
            duplicate=False
      schedule.append(temp)
   
       
    schedulestring=""
    for e in schedule:
      schedulestring += e  
    population.append(Individual(schedulestring, -1))
  return population
  
new_population=generatePopulation(10,coursesDF, classes, teachersDF)
print(new_population)

for s in new_population:
  exams=getExamStrings(s.chromosome)
  print("*****************Schedule Option*******************")
  for e in exams:
    print(convertToExamObject(e))




[(Chromosome: 00000110000100000100100100100010110000000000000110000101000100100011010011110000000000000110001010000111000001100111000000000100001110000100000111000001100111101000000100001110001010000100100011100010101000000000001110000001000111000010110110100000000100001000001001000111000010110100100000000100010000000001000100100101010010100000000100000100000101000111000101010110101000000100001010001001000100100000010000100000000100000010000100000100100000010000110000000100001100000101000111000011000101111000000000001100000100000111000001110110100000000100001100000110000111000001110011111000000100000010001001000111000100110110111000000100000100000100000111000100110101000000000100000110000100000111000000000111000000000100000000001001000100100000000011011000000100010010001000000100100010000001010000000000000000000101000100100100000001110000000100001110000001000100100100000001101000000100000110000101000100100010100011100000000100001010001001000100100010100010011000000100001000000001000111

Roulette Wheel Selection to get the fittest parent based on its probability 

In [ ]:
#roulette wheel selection
def rouletteWheelSelection(population):
  totalFitness=sum([p.fitness for p in population])
  individualProbabilities=[p.fitness/totalFitness for p in population]

  return population[np.random.choice(len(population), p=individualProbabilities)]


Main fitness function that calculates the number of conflicts in the existing schedule and returns a fitness value calculated as the inverse of the number of conflicts

In [ ]:
def getFitness(schedule):
  noOfConflicts = 0
  sameTimeExams = 0
  missingExams = 0
  examTimes = 0
  weekendExams = 0
  sameTimeInvigilate = 0
  consecutiveInvigilate = 0
  fridayBreaks = 0
  mgExams=0
  twohrbreak=0


  listOfExamsStrings=getExamStrings(schedule.chromosome)
  listOfExams=[]
  for e in listOfExamsStrings:
    listOfExams.append(convertToExamObject(e))
  uniqueExams = {e.courseCode for e in listOfExams}  
  uniqueExams = list(uniqueExams)
  
  #check that all courses scheduled
  courseCodes=coursesDF["CourseCode"].tolist()
  if len(uniqueExams)==len(courseCodes): 
    for e in uniqueExams:
      for i in range(len(courseCodes)):
        if e==courseCodes[i]:
          courseCodes[i]=None
    if (courseCodes.count(None)==len(courseCodes)):
      missingExams=0
    else:
      missingExams+=len(courseCodes)-courseCodes.count(None)
  else:
    missingExams+=abs(len(courseCodes)-len(uniqueExams))

  # Get the courses scheduled at the same time and day
  sameTimes=[[]]
  examsCopy = copy.deepcopy(listOfExams)
  for e,eobj in enumerate(examsCopy):
    day=eobj.time.day
    hour=eobj.time.starting_hour
    templist=[]
    for e2,e2obj in enumerate(examsCopy):      
      if (e2obj.time.day==day and e2obj.time.starting_hour==hour and e2obj.time.day!=-1):
        e2copyObj = copy.deepcopy(e2obj)
        templist.append(e2copyObj)
        examsCopy[e2].time.day = -1
    sameTimes.append(templist)
   

  # Get the list of students studying each course
  for i in range(len(sameTimes)):
    if (len(sameTimes[i])>1): #more than one exam at this time    
      students=[[]]
      courses=[]
      for j in range (len(sameTimes[i])):
        stcourses=list(np.where(studentCourse['CourseCode']==sameTimes[i][j].courseCode)[0])
        templist=[]
        for c in stcourses:
          templist.append(studentCourse.at[c, 'StudentName'])
        students.append(templist)
        courses.append(sameTimes[i][j].courseCode)


      # Check for number of students with the same exam time scheduled
      for row in range(len(students)):
        for column in range(len(students[row])):
          name=students[row][column]
          for row2 in range(len(students)):
            for column2 in range(len(students[row2])):
              if name==students[row2][column2] and row2!=row and name!="xyz":
                # xyz garbage value, to represent already checked students
                name="xyz"
                students[row][column]="xyz"
                sameTimeExams+=1  

  # No teacher invigilating exam at the same time
  sameTimes2 = copy.deepcopy(sameTimes)
  for i,row in enumerate(sameTimes2):
    for j,col in enumerate(sameTimes2[i]):
      curr_teacher = sameTimes2[i][j].invigilator
      for k,cols in enumerate(sameTimes2[i]):
        if sameTimes2[i][k].invigilator == curr_teacher and sameTimes2[i][k].invigilator!='xyz' and k!=j and sameTimes2[i][j].invigilator!='xyz':
          # xyz garbage value, to represent already checked teachers
          sameTimes2[i][k].invigilator = 'xyz'
          sameTimes2[i][j].invigilator = 'xyz'  
          sameTimeInvigilate+=1

  # get a list of sorted exams based on the invigilator, the day and time
  sortedExams = sorted(listOfExams, key=lambda x: (x.invigilator,x.time.day, x.time.starting_hour))
   
  for i in range(len(sortedExams)-1):
    # Consecutive Exams means invigilator can not conduct an exam consecutively on the same DAY and different Time
    if sortedExams[i].invigilator == sortedExams[i+1].invigilator and sortedExams[i].time.day == sortedExams[i+1].time.day and sortedExams[i].time.starting_hour != sortedExams[i+1].time.starting_hour:
      consecutiveInvigilate+=1


  # Soft Constraint # 1. Firday Breaks between 1-2pm
  for e in listOfExams:
    Day = e.time.day
    Time = e.time.starting_hour
    if Day%5 == 0 and Time > 10 and Time < 14:
      firdayBreaks += 1

  #Soft Constraint # 3. No CS exam before MG exam for student
  MGCoursesChecked=[]
  CSCoursesChecked=[]
  for e in listOfExams:
    leaveloop=0
    if e.courseCode[0:2]=='MG' and  e.courseCode not in MGCoursesChecked:
      for e2 in listOfExams:
        if e2.courseCode[0:2]!='MG' and (e2.time.day<e.time.day or (e2.time.day==e.time.day and e2.time.starting_hour<e.time.starting_hour)):
          csstudentsi=list(np.where(studentCourse['CourseCode']==e2.courseCode)[0])
          csstudents=list()
          for abc in csstudentsi:
            csstudents.append(studentCourse.at[abc,'StudentName'])

          mgstudentsi=list(np.where(studentCourse['CourseCode']==e.courseCode)[0])   
          mgstudents=list()
          for abc in mgstudentsi:       
            mgstudents.append(studentCourse.at[abc,'StudentName'])

          #if even one value is same in both
          for cs in csstudents:
            for mg in mgstudents:
              if (cs==mg):
                leaveloop=1
                break
              if leaveloop==1:
                break
          if leaveloop==1:
           break
      if (leaveloop==1):
        MGCoursesChecked.append(e.courseCode)
        mgExams+=0.1

  #check if any teacher busy in two slots at once so violating condition for break
  teachersChecked=[]
  for e in listOfExams:
    t=e.invigilator
    leave=0
    if t not in teachersChecked:
      for e2 in listOfExams:
        if e2.invigilator==t and e2.time.starting_hour != e.time.starting_hour:
          leave=1
          break
      if (leave==1):
        teachersChecked.append(t)
        twohrbreak+=0.1
        

  # Extra Constraint, no exam should have more than its scheduled multiple exams
  RoomsCheckedCourses = []
  CorrectMultipleRooms = 0
  for e in listOfExams:
    crsCode = e.courseCode
    multExams = e.multipleRooms
    count = 0
    if crsCode not in RoomsCheckedCourses:
      for e2 in listOfExams:
        if crsCode == e2.courseCode and e!=e2:
          count += 1
          if count < multExams:
            multExams = abs(multExams - count)
      RoomsCheckedCourses.append(e.courseCode)
      CorrectMultipleRooms += multExams
  
  #noOfConflicts += CorrectMultipleRooms

  #check that no exam before 9 or after 5
  examTiming=0
  for e in listOfExams:
    if (e.time.starting_hour<9 or e.time.starting_hour>14):
      examTiming+=1

  #check that no exam on weekends
  for e in listOfExams:
    if getDay(e.time.day)=="Saturday" or getDay(e.time.day)=="Sunday":
      weekendExams+=1 


  noOfConflicts+=missingExams
  schedule.h1=missingExams

  noOfConflicts+=sameTimeExams
  schedule.h2=sameTimeExams

  noOfConflicts+=weekendExams
  schedule.h3=weekendExams

  noOfConflicts+=examTiming
  schedule.h4=examTiming

  noOfConflicts+=sameTimeInvigilate
  schedule.h5=sameTimeInvigilate

  noOfConflicts+=consecutiveInvigilate
  schedule.h6=consecutiveInvigilate

  noOfConflicts += fridayBreaks
  schedule.s1 = fridayBreaks

  noOfConflicts+=mgExams
  schedule.s3=mgExams

  noOfConflicts+=twohrbreak
  schedule.s4=twohrbreak

  schedule.fitness= 1/(float(noOfConflicts)+1)
  return noOfConflicts




Main (multi point) Crossover function that crosses the exams from both the parents and returns one offspring

In [ ]:
def crossover(best, second_best):
  crossed_exams_two= []
  crossed_exams_one = []

  # Size of one exam is = 42 bits
  exams_one = getExamStrings(best.chromosome)
  exams_two = getExamStrings(second_best.chromosome)
  

  for i,val in enumerate(exams_one):    
#     #0-7 bits for class, 7-14 bits for day, 14-21 bit for starting hour, 21-28 bits for course, 28-35 bits for teacher, 35-42 bits for multiple rooms
    # For even iterations
    if (i%2==0):
      exam_cross_one =val[0:7] + exams_two[i][7:14] + val[14:21] + exams_two[i][21:28] + val[28:35] + exams_two[i][35:42] 
    # For odd iterations
    else:
      exam_cross_one = exams_two[i][0:7] + val[7:14] + exams_two[i][14:21] + val[21:28] + exams_two[i][28:35] + val[35:42]

    crossed_exams_one.append(exam_cross_one)
    
  crossover_schedule=Individual("", -1) 
  crossover_schedule.chromosome=""


  for x in crossed_exams_one:
    crossover_schedule.chromosome+=x

  return crossover_schedule


Main mutation function that generates new exams based on the mutation probability.
Mutation probability is kept as 0.2 because too much mutation will create unusual ambiguities that aren't too preferable but obviously some mutation is helpful

In [ ]:
def mutation(best,mutation_probability):
  
  exams_one=getExamStrings(best.chromosome)
  
  for i in range(len(exams_one)):
    if (mutation_probability>random.random()):
      exams_one[i]=convertToChromosome(exams_one[i]);
    
  best.chromosome=""
  for e in exams_one:
    best.chromosome+=e
    
  return best



Sort population based on fitness

In [ ]:
def getSortedFitness(population):
  population=sorted(population, key=lambda x: (x.fitness))
  return population

Crossover population function which creates new chromosomes based on parents selected by roulette wheel selection and then crossed to make a new schedule.
This repeats till a new population, with the same size, is generated. 

In [ ]:
def crossover_population(population, size):
  population=getSortedFitness(population)
  new_crossed_population=[]
  new_crossed_population.append(population[-1]) #append the schedule with the highest fitness
  i=1
  while (i<size):
    fittest=copy.deepcopy(rouletteWheelSelection(population))
    second_fittest=copy.deepcopy(rouletteWheelSelection(population))
    new_schedule=crossover(fittest, second_fittest)
    getFitness(new_schedule)
    # add the new crossed schedule only if it has a better fitness value
    if new_schedule.fitness > fittest.fitness and new_schedule.fitness > second_fittest.fitness:
      # check if extra exams have been created for any course then perform crossover again
      exams = getExamStrings(new_schedule.chromosome)
      makeCont = 0
      for e in exams:
        examObj = convertToExamObject(e)
        req = examObj.multipleRooms
        countRooms = 0
        for e2 in exams:
          examObj2 = convertToExamObject(e2)
          # req to have another room
          if e != e2 and req > 0:
            if examObj.courseCode == examObj2.courseCode:
              countRooms += 1
        if countRooms != req:
          makeCont = 1
          break
      if makeCont:
        continue

      new_crossed_population.append(new_schedule)
    elif fittest.fitness > new_schedule.fitness and fittest.fitness > second_fittest.fitness:
      new_crossed_population.append(fittest)
    else:
      new_crossed_population.append(second_fittest)
    i+=1
  return new_crossed_population


Mutate population function that calls mutation over the entire population

In [ ]:
def mutate_population(population, size, prob):
  for i in range (1, size):
    current_mutated = copy.deepcopy(population[i])
    current_mutated=mutation(current_mutated,prob)
    getFitness(current_mutated)
    population[i] = copy.deepcopy(current_mutated)
  return population


Propogate the population by performing crossovers and then mutations

In [ ]:
def propogate(population, size, prob):
  return mutate_population(crossover_population(population, size), size, prob)

## **Implementation of GA Algorithm**

Main Genetic Algorithm function which generates a population and then for Max Generations propgates and attempts to find an optimum schedule with fitness value = 1.0 

In [ ]:
def GA(size):
  mutation_prob=0.2
  hardconstraints=0
  soft=0
  population=generatePopulation(size, coursesDF, classes, teachersDF)
  for p in population:
    getFitness(p)
  print("The maximum fitness at start is ", population[-1].fitness)
  for i in range (1000):
    population=propogate(population, size, mutation_prob)
    population=getSortedFitness(population)
    print("The maximum fitness uptil now is ", population[-1].fitness)
    if (population[-1].fitness>0.5 and hardconstraints==0): #hard constraints satisifed
      print("\U0001f600", "Schedule with all hard constraints satisfied:")
      print("Chromosome: ", population[-1].chromosome)    
      exams=getExamStrings(population[-1].chromosome)
      examList=[]
      for e in exams:
        examList.append(convertToExamObject(e))
      sortedExams = sorted(examList, key=lambda x: (x.time.day, x.time.starting_hour))
      for e in sortedExams:
        print(e)
      hardconstraints=1 #mark hard constraints as satisfied
      soft+=1
      print("List of hard constraints satisifed: ")
      if (population[-1].h1==0):
        print("An exam will be scheduled for each course ", "\U00002714")
      if (population[-1].h2==0):
        print("A student cannot give more than one exam at a time ", "\U00002714")
      if (population[-1].h3==0):
        print("Exam will not be held on weekends", "\U00002714")
      if (population[-1].h4==0):
        print("Exam must be held between 9am and 5pm", "\U00002714")
      if (population[-1].h5==0):
        print("Teacher cannot invigilate two exams at the same time", "\U00002714")
      if (population[-1].h6==0):
        print("Teacher cannot invigilate two consecutive exams", "\U00002714")

    if (hardconstraints==1):#hard constraints have been satisfied
      soft+=1
    if (population[-1].fitness==1 or soft>=50): #most soft constraints satisfied as well
      break;
 
  return population[-1] 


# **Output Examples on Large DataSet**

In [ ]:
# Testing with output
schedule = GA(100)
print("\U0001f600", "The most optimum schedule: **")
print("Chromosome: ", schedule.chromosome) 
exams=getExamStrings(schedule.chromosome)
examList=[]
for e in exams:
  examList.append(convertToExamObject(e))
sortedExams = sorted(examList, key=lambda x: (x.time.day, x.time.starting_hour))
for e in sortedExams:
  print(e)
print("List of hard constraints satisifed: ")
if (schedule.h1==0):
  print("An exam will be scheduled for each course ", "\U00002714")
if (schedule.h2==0):
  print("A student cannot give more than one exam at a time ", "\U00002714")
if (schedule.h3==0):
  print("Exam will not be held on weekends", "\U00002714")
if (schedule.h4==0):
  print("Exam must be held between 9am and 5pm", "\U00002714")
if (schedule.h5==0):
  print("Teacher cannot invigilate two exams at the same time", "\U00002714")
if (schedule.h6==0):
  print("Teacher cannot invigilate two consecutive exams", "\U00002714")


print("List of soft constraints satisifed: ")
if (schedule.s1==0):
  print("Break on Friday 1-2", "\U00002714")
if (schedule.s2==0):
  print("No more than 1 consecutive exam for students ", "\U00002714")
if (schedule.s3==0):
  print("All students MG Course be held before their CS", "\U00002714")
if (schedule.s4==0):
  print("Half faculty free in one slot in week and half in other", "\U00002714")


The maximum fitness at start is  0.004522840343735866
The maximum fitness uptil now is  0.026954177897574122
The maximum fitness uptil now is  0.47619047619047616
The maximum fitness uptil now is  0.9090909090909091
😀 Schedule with all hard constraints satisfied:
Chromosome:  000001100000010001110000001001011110000001000100000001000001110000001001001110000001000100100000010001001000100100100000000000000001100000110001110001010001001110000001000001000000110001001001010000111010000001000000000000110001110000000101010110000001000100000010010001001000000100000110000001000100100010010001001000011100100000000001000010000000010001001000011100000000000001000011100001110001110000000001001010000001000010000010000001110000000001001110000001000010100001100001001000100000000110000000000100000001010001110000111001011010000000000010000000100001001001001000011010000000000100100000110001110001000001010000000001000001000001010001001001000000001110000001000000000000100001001000110000010000000000000100100

In [ ]:
# Testing with output
schedule = GA(50)
print("\U0001f600", "The most optimum schedule: **")
print("Chromosome: ", schedule.chromosome) 
exams=getExamStrings(schedule.chromosome)
examList=[]
for e in exams:
  examList.append(convertToExamObject(e))
sortedExams = sorted(examList, key=lambda x: (x.time.day, x.time.starting_hour))
for e in sortedExams:
  print(e)
print("List of hard constraints satisifed: ")
if (schedule.h1==0):
  print("An exam will be scheduled for each course ", "\U00002714")
if (schedule.h2==0):
  print("A student cannot give more than one exam at a time ", "\U00002714")
if (schedule.h3==0):
  print("Exam will not be held on weekends", "\U00002714")
if (schedule.h4==0):
  print("Exam must be held between 9am and 5pm", "\U00002714")
if (schedule.h5==0):
  print("Teacher cannot invigilate two exams at the same time", "\U00002714")
if (schedule.h6==0):
  print("Teacher cannot invigilate two consecutive exams", "\U00002714")


print("List of soft constraints satisifed: ")
if (schedule.s1==0):
  print("Break on Friday 1-2", "\U00002714")
if (schedule.s2==0):
  print("No more than 1 consecutive exam for students ", "\U00002714")
if (schedule.s3==0):
  print("All students MG Course be held before their CS", "\U00002714")
if (schedule.s4==0):
  print("Half faculty free in one slot in week and half in other", "\U00002714")


The maximum fitness at start is  0.006169031462060457
The maximum fitness uptil now is  0.04739336492890995
The maximum fitness uptil now is  0.04739336492890995
The maximum fitness uptil now is  0.04739336492890995
The maximum fitness uptil now is  0.04739336492890995
The maximum fitness uptil now is  0.04739336492890995
The maximum fitness uptil now is  0.5
The maximum fitness uptil now is  1.0
😀 Schedule with all hard constraints satisfied:
Chromosome:  00000110001001000111000000100111100000000100000110000001000111000000100101101000000100000100001001000100100010110000000000000100000100000110000111000010110110101000000100001100000011000100100001100010101000000100001100001000000100100001100010000000000100001100000110000100100101000010010000000100000000000100000100100101000011110000000100010010000101000111000010000100010000000000010010000011000100100010010000001000000000010010000111000111000010100110101000000100000110000010000111000010100110000000000100010010001000000111000001010100101

In [ ]:
# Testing with output
schedule = GA(25)
print("\U0001f600", "The most optimum schedule: **")
print("Chromosome: ", schedule.chromosome) 
exams=getExamStrings(schedule.chromosome)
examList=[]
for e in exams:
  examList.append(convertToExamObject(e))
sortedExams = sorted(examList, key=lambda x: (x.time.day, x.time.starting_hour))
for e in sortedExams:
  print(e)
print("List of hard constraints satisifed: ")
if (schedule.h1==0):
  print("An exam will be scheduled for each course ", "\U00002714")
if (schedule.h2==0):
  print("A student cannot give more than one exam at a time ", "\U00002714")
if (schedule.h3==0):
  print("Exam will not be held on weekends", "\U00002714")
if (schedule.h4==0):
  print("Exam must be held between 9am and 5pm", "\U00002714")
if (schedule.h5==0):
  print("Teacher cannot invigilate two exams at the same time", "\U00002714")
if (schedule.h6==0):
  print("Teacher cannot invigilate two consecutive exams", "\U00002714")


print("List of soft constraints satisifed: ")
if (schedule.s1==0):
  print("Break on Friday 1-2", "\U00002714")
if (schedule.s2==0):
  print("No more than 1 consecutive exam for students ", "\U00002714")
if (schedule.s3==0):
  print("All students MG Course be held before their CS", "\U00002714")
if (schedule.s4==0):
  print("Half faculty free in one slot in week and half in other", "\U00002714")


The maximum fitness at start is  0.003757985719654265
The maximum fitness uptil now is  0.023752969121140142
The maximum fitness uptil now is  0.04524886877828054
The maximum fitness uptil now is  0.04524886877828054
The maximum fitness uptil now is  0.04524886877828054
The maximum fitness uptil now is  0.04524886877828054
The maximum fitness uptil now is  0.47619047619047616
The maximum fitness uptil now is  0.47619047619047616
The maximum fitness uptil now is  0.47619047619047616
The maximum fitness uptil now is  0.47619047619047616
The maximum fitness uptil now is  0.9090909090909091
😀 Schedule with all hard constraints satisfied:
Chromosome:  000000000010100001001000101000000000000001000011000000100001110000101001010000000001000001000001110001110000100001011110000000000100100001100001110000110001000000000000000000000000100001001001001000101100000000000000000010010001110001000101101000000001000011100000010001001001000100101010000001000000000001000001110000100101011100000000000001000

# **Best Output**

In [ ]:
schedule = GA(25)
print("\U0001f600", "The most optimum schedule: **")
print("Chromosome: ", schedule.chromosome) 
exams=getExamStrings(schedule.chromosome)
examList=[]
for e in exams:
  examList.append(convertToExamObject(e))
sortedExams = sorted(examList, key=lambda x: (x.time.day, x.time.starting_hour))
for e in sortedExams:
  print(e)
print("List of hard constraints satisifed: ")
if (schedule.h1==0):
  print("An exam will be scheduled for each course ", "\U00002714")
if (schedule.h2==0):
  print("A student cannot give more than one exam at a time ", "\U00002714")
if (schedule.h3==0):
  print("Exam will not be held on weekends", "\U00002714")
if (schedule.h4==0):
  print("Exam must be held between 9am and 5pm", "\U00002714")
if (schedule.h5==0):
  print("Teacher cannot invigilate two exams at the same time", "\U00002714")
if (schedule.h6==0):
  print("Teacher cannot invigilate two consecutive exams", "\U00002714")


print("List of soft constraints satisifed: ")
if (schedule.s1==0):
  print("Break on Friday 1-2", "\U00002714")
if (schedule.s2==0):
  print("No more than 1 consecutive exam for students ", "\U00002714")
if (schedule.s3==0):
  print("All students MG Course be held before their CS", "\U00002714")
if (schedule.s4==0):
  print("Half faculty free in one slot in week and half in other", "\U00002714")

The maximum fitness at start is  0.0030021014710297205
The maximum fitness uptil now is  0.010976948408342482
The maximum fitness uptil now is  0.016366612111292964
The maximum fitness uptil now is  0.016366612111292964
The maximum fitness uptil now is  0.016366612111292964
The maximum fitness uptil now is  0.9090909090909091
😀 Schedule with all hard constraints satisfied:
Chromosome:  00001000000110000100100010010000001000000000010000000011000100100010110010110000000100000010001010000100100010110001000000000100001010001000000111000000010111001000000100000000001000000100100000010011000000000100000000001001000100100000100010100000000100010010001010000111000000100110000000000100010010001000000111000001100100000000000100000100000010000100100001100001010000000100001010000111000111000011100100001000000000001110000110000111000100100100000000000000000110000001000100100001000001111000000100001010000011000100100001000001010000000100000100000110000111000101000101001000000100000010000100000100100